Notebook to backtest with backtrader the ML scores built for the POC about the DAX in timeframe M30.

In [ ]:
from importlib import reload
import pandas as pd
import backtrader as bt
import bt_ml_strategy
from bt_ml_strategy import SignalData,BTMLStrategy
reload(bt_ml_strategy)

PATH_DATA_Quant = "../../Data/dl_quant/"



In [ ]:
df_dax_m30 = pd.read_csv(PATH_DATA_Quant+"DAX_M30_D1_with_predict.zip", sep=",",
                            index_col="Datetime", parse_dates=True, keep_default_na=True, low_memory=False)

In [ ]:
prices=df_dax_m30[['Open','High','Low','Close','Volume','meta_score_rf_xgb','m30_val_sl','m30_tp_3sl']]
prices.rename(columns={'Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume','m30_val_sl':'SL','m30_tp_3sl':'TP','meta_score_rf_xgb':'Predict'},inplace=True)
prices=prices['2020-07-01 00:00:00':]
prices.dropna(inplace=True)
prices.sort_index(inplace=True)
#prices.info()

In [ ]:
data = SignalData(dataname=prices,timeframe=bt.TimeFrame.Minutes, compression=30)

In [ ]:
cerebro = bt.Cerebro(cheat_on_open=True)
cerebro.addstrategy(BTMLStrategy)
cerebro.adddata(data)
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.002)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")

print(f"Starting portfolio:{cerebro.broker.getvalue()}")
backtest_result=cerebro.run()
print(f"Ending portfolio:{cerebro.broker.getvalue()}")